# 🧠 Unified Classification + Reasoning (FLAN‑T5) — Kaggle Training Notebook

> This notebook fine-tunes a FLAN‑T5 model to output both a fraud label and a short explanation in a single generation, using your compiled dataset.

It mirrors `training/unified_t5_fraud.py` but is optimized for Kaggle. Make sure your CSV is provided as a Kaggle dataset input (e.g., `/kaggle/input/fraud-data/final_fraud_detection_dataset.csv`).

In [1]:
# 1) Detect Kaggle Environment and Set Paths
import os, glob, json, sys
from pathlib import Path

IS_KAGGLE = Path('/kaggle').exists()
INPUT_DIR = Path('/kaggle/input') if IS_KAGGLE else Path('..')
WORK_DIR = Path('/kaggle/working') if IS_KAGGLE else Path('.')
WORK_DIR.mkdir(parents=True, exist_ok=True)

print(f"Running on Kaggle: {IS_KAGGLE}")
print(f"INPUT_DIR: {INPUT_DIR}")
print(f"WORK_DIR: {WORK_DIR}")

if IS_KAGGLE:
    print("\nAvailable datasets under /kaggle/input:")
    for p in sorted(INPUT_DIR.glob('*')):
        print(' -', p)

Running on Kaggle: True
INPUT_DIR: /kaggle/input
WORK_DIR: /kaggle/working

Available datasets under /kaggle/input:
 - /kaggle/input/fraud-detection-dataset


In [2]:
# 2) Import and (Optionally) Install Libraries
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss
from pathlib import Path

import torch
import transformers as tf
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Trainer,
    TrainingArguments,
    set_seed,
)
import inspect

print('PyTorch:', torch.__version__)
print('Transformers:', tf.__version__)
device = 'cuda' if torch.cuda.is_available() else ('mps' if hasattr(torch.backends, 'mps') and torch.backends.mps.is_available() else 'cpu')
print('Device:', device)

2025-09-19 09:43:18.670387: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758274998.882081      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758274998.944749      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


PyTorch: 2.6.0+cu124
Transformers: 4.52.4
Device: cuda


In [3]:
# 3) Configuration (Seeds, Paths, Target, Problem Type)
SEED = 42
set_seed(SEED)

# Point to your fraud dataset CSV under /kaggle/input
# Adjust this path based on your dataset name
CSV_PATH = INPUT_DIR / 'fraud-data' / 'final_fraud_detection_dataset.csv'
if not CSV_PATH.exists():
    # Fallback guess: sometimes users rename the dataset folder
    matches = list(INPUT_DIR.glob('**/final_fraud_detection_dataset.csv'))
    if matches:
        CSV_PATH = matches[0]
print('CSV_PATH:', CSV_PATH)

TEXT_COL = 'text'
LABEL_COL = 'detailed_category'

DEFAULT_LABELS = [
    'job_scam',
    'legitimate',
    'phishing',
    'popup_scam',
    'refund_scam',
    'reward_scam',
    'sms_spam',
    'ssn_scam',
    'tech_support_scam'
]

MODEL_NAME = 'google/flan-t5-small'
OUTPUT_DIR = WORK_DIR / 'unified-flan-t5-small'
MAX_SOURCE_LENGTH = 256
MAX_TARGET_LENGTH = 64
TRAIN_SIZE = 0.9
NUM_EPOCHS = 3
LR = 3e-4
BATCH_TRAIN = 8
BATCH_EVAL = 8
GRAD_ACCUM = 1

INSTRUCTION_PREFIX = (
    "Classify the message into one of these categories and explain briefly:\n"
    "Categories: job_scam, legitimate, phishing, popup_scam, refund_scam, reward_scam, sms_spam, ssn_scam, tech_support_scam\n"
    "Message: "
)

CSV_PATH: /kaggle/input/fraud-detection-dataset/final_fraud_detection_dataset.csv


In [4]:
# 4) Load Data from /kaggle/input
assert CSV_PATH.exists(), f"CSV not found at {CSV_PATH}. Attach your dataset to the notebook."
df = pd.read_csv(CSV_PATH)
print('Loaded:', df.shape)
print(df.head(3))

# Filter to known labels only
df = df[df[LABEL_COL].isin(DEFAULT_LABELS)].copy()
print('After label filtering:', df.shape)

# 5) Basic Data Checks
assert TEXT_COL in df.columns and LABEL_COL in df.columns, 'Missing required columns'
print('Columns:', list(df.columns))
print('Label distribution:')
print(df[LABEL_COL].value_counts())

# Train/Validation split (stratified)
train_df, val_df = train_test_split(df, test_size=1-TRAIN_SIZE, random_state=SEED, stratify=df[LABEL_COL])
print('Train:', train_df.shape, 'Val:', val_df.shape)

Loaded: (194913, 4)
                                                text  binary_label  \
0  Position Summary The Asset Manager will plan, ...             1   
1  We offer interns that can develop web sites re...             1   
2  We are a Health Benefits company. Helping peop...             1   

  detailed_category            data_type  
0          job_scam  text_classification  
1          job_scam  text_classification  
2          job_scam  text_classification  
After label filtering: (194913, 4)
Columns: ['text', 'binary_label', 'detailed_category', 'data_type']
Label distribution:
detailed_category
legitimate           101717
phishing              71857
popup_scam            11333
sms_spam               6988
reward_scam             606
tech_support_scam       605
refund_scam             604
ssn_scam                604
job_scam                599
Name: count, dtype: int64
Train: (175421, 4) Val: (19492, 4)


In [5]:
# 6) Build Targets: label + short reason
def default_reason_for_label(label: str) -> str:
    templates = {
        'legitimate': "Message content appears normal and lacks scam indicators.",
        'phishing': "Contains credential-stealing cues (links/requests for login or personal info).",
        'tech_support_scam': "Mentions fake support, urgent fixes, or remote access.",
        'reward_scam': "Promises winnings/prizes with urgency or fees.",
        'job_scam': "Unrealistic job offers, upfront payments, or unsolicited interviews.",
        'sms_spam': "Unwanted promotional or bulk messaging characteristics.",
        'popup_scam': "Fake security alerts/popups demanding immediate action.",
        'refund_scam': "Unexpected refund/chargeback claims with links or callbacks.",
        'ssn_scam': "Requests social security info or threats about your SSN.",
    }
    return templates.get(label, "Heuristic cues indicate this category.")

def build_target(label: str, text: str, reason: str = None) -> str:
    if not reason or not isinstance(reason, str) or len(reason.strip()) == 0:
        reason = default_reason_for_label(label)
    return f"label: {label} | reason: {reason}"

def make_examples(df, text_col: str, label_col: str):
    examples = []
    for _, row in df.iterrows():
        text = str(row[text_col])
        label = str(row[label_col])
        examples.append({
            'source': f"{INSTRUCTION_PREFIX}{text}",
            'target': build_target(label, text),
        })
    return examples

train_examples = make_examples(train_df, TEXT_COL, LABEL_COL)
val_examples = make_examples(val_df, TEXT_COL, LABEL_COL)
print('Examples:', len(train_examples), len(val_examples))

Examples: 175421 19492


In [6]:
# 7) Tokenizer and Simple Dataset Wrappers
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)
model.to(device)

class SimpleMapDataset:
    def __init__(self, examples):
        self.examples = examples
    def __len__(self):
        return len(self.examples)
    def __getitem__(self, idx):
        e = self.examples[idx]
        model_inputs = tokenizer(
            e['source'],
            max_length=MAX_SOURCE_LENGTH,
            truncation=True,
            padding='max_length',
)
        with tokenizer.as_target_tokenizer():
            labels = tokenizer(
                e['target'],
                max_length=MAX_TARGET_LENGTH,
                truncation=True,
                padding='max_length',
)
        model_inputs['labels'] = labels['input_ids']
        return model_inputs

train_ds = SimpleMapDataset(train_examples)
val_ds = SimpleMapDataset(val_examples)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [7]:
# 8) TrainingArguments, Trainer, and Metrics
from typing import Any

def extract_label_from_text(text: str) -> str:
    t = " ".join(str(text).strip().lower().split())
    if 'label:' in t:
        after = t.split('label:', 1)[1].strip()
        if '|' in after:
            after = after.split('|', 1)[0].strip()
        return " ".join(after.split()[:5])
    return 'unknown'


def compute_metrics(eval_pred: Any):
    # Support both (preds, labels) tuple and EvalPrediction object
    preds = getattr(eval_pred, 'predictions', None)
    labels_ids = getattr(eval_pred, 'label_ids', None)
    if preds is None or labels_ids is None:
        preds, labels_ids = eval_pred
    pred_texts = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels_ids = [[(lid if lid != -100 else tokenizer.pad_token_id) for lid in seq] for seq in labels_ids]
    gold_texts = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    pred_labels = [extract_label_from_text(t) for t in pred_texts]
    gold_labels = [extract_label_from_text(t) for t in gold_texts]
    canon = set(DEFAULT_LABELS)
    def canonize(s: str) -> str:
        s = s.strip()
        if s in canon:
            return s
        for c in canon:
            if s in c or c in s:
                return c
        return s
    pred_labels = [canonize(x) for x in pred_labels]
    gold_labels = [canonize(x) for x in gold_labels]
    acc = accuracy_score(gold_labels, pred_labels)
    return {'label_accuracy': acc}

# Build TrainingArguments robustly against Transformers version changes
args_dict = {
    'output_dir': str(OUTPUT_DIR),
    'learning_rate': LR,
    'per_device_train_batch_size': BATCH_TRAIN,
    'per_device_eval_batch_size': BATCH_EVAL,
    'num_train_epochs': NUM_EPOCHS,
    'weight_decay': 0.01,
    'warmup_ratio': 0.05,
    'logging_steps': 50,
    'gradient_accumulation_steps': GRAD_ACCUM,
    'report_to': ['none'],
    'fp16': torch.cuda.is_available(),
    'bf16': False,
}

sig = inspect.signature(TrainingArguments.__init__)
if 'evaluation_strategy' in sig.parameters:
    args_dict['evaluation_strategy'] = 'epoch'
if 'save_strategy' in sig.parameters:
    args_dict['save_strategy'] = 'epoch'
if 'predict_with_generate' in sig.parameters:
    args_dict['predict_with_generate'] = True
if 'generation_max_length' in sig.parameters:
    args_dict['generation_max_length'] = MAX_TARGET_LENGTH

training_args = TrainingArguments(**args_dict)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [8]:
# 9) Train
trainer.train()
trainer.save_model(str(OUTPUT_DIR))
tokenizer.save_pretrained(str(OUTPUT_DIR))
print('Model saved to:', OUTPUT_DIR)

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
50,31.709200
100,20.291800
150,10.135700
200,4.296000
250,2.343400
300,1.213800
350,0.497300
400,0.171700
450,0.074800
500,0.040300


Model saved to: /kaggle/working/unified-flan-t5-small


In [9]:
# 10) Quick Sanity Generation
model.eval()
sample_text = val_df.iloc[0][TEXT_COL] if len(val_df) else train_df.iloc[0][TEXT_COL]
inputs = tokenizer([f"{INSTRUCTION_PREFIX}{sample_text}"], return_tensors='pt', truncation=True, padding=True).to(model.device)
with torch.no_grad():
    gen = model.generate(**inputs, max_new_tokens=64, do_sample=False)
print('Input snippet:', sample_text[:120])
print('Output:', tokenizer.decode(gen[0], skip_special_tokens=True))

Input snippet: webmailservicestratusroutersailinenet dear josemonkeyorg due error imappop settings access josemonkeyorg required verify
Output: label: phishing | reason: Contains credential-stealing cues (links/requests for login or personal info).


In [10]:
# 11) Zip Artifacts (Optional)
import shutil
ZIP_PATH = WORK_DIR / 'unified-flan-t5-small.zip'
if ZIP_PATH.exists():
    ZIP_PATH.unlink()
shutil.make_archive(str(ZIP_PATH.with_suffix('')), 'zip', str(OUTPUT_DIR))
print('Zipped model to:', ZIP_PATH)

Zipped model to: /kaggle/working/unified-flan-t5-small.zip
